# P18 URLCategorization
***
### Authors: Asier Barrasa and Iván Valdés 



## Objectives
The aim of this project is to clasiffy a set of tweets among 5 diferent classess according to the sentiment toward self-driving cars.  

## What is done in the Notebook: 

In this notebook, first we preprocess the given data-set int a cleaned version of it. Then we implement two NN's to solve the problem and finnaly, we compute the accuracy. 

In [2]:
import sys
import multiprocessing
import warnings
from tqdm import tqdm

warnings.filterwarnings('ignore')

import numpy as np
import tensorflow as tf 
import pandas as pd
from time import time

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

from sklearn import utils
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer , CountVectorizer
from sklearn.metrics import accuracy_score

# imports for doc2vec
import gensim
from gensim.models import Doc2Vec
from gensim.models.doc2vec import LabeledSentence

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Asier\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Define the functions

In [60]:
# function for cleanning tweets
def clean_tweets(data_set):
    for index in range(len(data_set['text'])):
        if index % 100 ==0:
            print(index)
        if index==409:
            print("ahora")
        if(pd.isna(data_set['text'][index]) ):
            print("el tweet es " + data_set['text'][index] + " de indice "+ str(index))
            data_set.drop(data_set.index[index], inplace=True)
        else:  
            tweet= data_set['text'][index]
            cleaned_tweet=clean_tweet(tweet)
            #print(cleaned_tweet)
            data_set['text'][index] = cleaned_tweet
        data_set.drop(data_set[df2.text.isna()].index, inplace=True) #Remove empty cells
    return data_set

# funcion for clean each tweet
def clean_tweet(tweet):
    # Convert text to lower
    tweet = tweet.lower()
    #Remove urls
    regex_url = re.compile('(http:\/\/www\.|https:\/\/www\.|http:\/\/|https:\/\/)?[a-z0-9]+([\-\.]{1}[a-z0-9]+)*\.[a-z]{2,5}(:[0-9]{1,5})?(\/.*)')
    tweet= re.sub(regex_url,'',tweet)
    # Removing non ASCII chars
    tweet = re.sub(r'[^\x00-\x7f]', r' ', tweet)
    # Remove the punctuation
    tweet = gensim.parsing.preprocessing.strip_punctuation2(tweet)
    # Strip all the numerics
    tweet = gensim.parsing.preprocessing.strip_numeric(tweet)
    # Strip multiple whitespaces
    tweet = gensim.corpora.textcorpus.strip_multiple_whitespaces(tweet)
    words= tweet.split()
    # Remove the english stopwords.Before this call, it is necessary to call to the function nltk.download('stopwords')
    stops = set(stopwords.words("english"))
    words = [word for word in words if word not in stopwords.words('english')]

    # Remove the words with less than 3 letters
    words = gensim.corpora.textcorpus.remove_short(words, minsize=3)
    return (' '.join(words))

# function for printing stats
def printStats(estimator,name,x_train, y_train, x_test, y_test):
    print(82 * '_')
    t0=time()
    if len(x_test[y_test == 0]) / (len(x_test)*1.) > 0.5:
        null_accuracy = len(x_test[y_test == 0]) / (len(x_test)*1.)
    else:
        null_accuracy = 1. - (len(x_test[y_test == 0]) / (len(x_test)*1.))
        t0 = time()
        sentiment_fit = estimator.fit(x_train, y_train)
        y_pred = sentiment_fit.predict(x_test)
        train_test_time = time() - t0
        accuracy = accuracy_score(y_test, y_pred)
        print('name\t\ttime\taccuracy')
        print('%-9s\t%.2fs\t%.2f\t'
              % (name, (time() - t0), (accuracy*100)))
       
    
def clean_data_set():    
    data_set = pd.read_csv('data-set.csv',encoding='latin1')
    cleaned_data_set = clean_tweets(data_set)
    df2 = cleaned_data_set[cleaned_data_set.sentiment!='not_relevant']
    #print(df[['sentiment','text']].sample(5))
    df2 = pd.DataFrame(df2[['text','sentiment']])
    df2.dropna()
    df2.to_csv('cleaned_data_set.csv',columns=['text','sentiment'])
    
def show_data_set_stats(df):
    
    print("Number of data by class")
    print("Class 1 -->", len(df[df.sentiment==1]))
    print("Class 2 -->",len(df[df.sentiment==2]))
    print("Class 3 -->",len(df[df.sentiment==3]))
    print("Class 4 -->",len(df[df.sentiment==4]))
    print("Class 5 -->",len(df[df.sentiment==5]))
    
# Transform the data on a one-hot vector
def oneHotVector(data, numClasses):
    targets = np.array(data).reshape(-1)
    data = np.eye(numClasses)[targets-1]
    return data

def labelize_tweets_ug(tweets,label):
    result = []
    prefix = label
    count = 0
    for t in  (tweets): 
        result.append(LabeledSentence(t.split(), [prefix + '_%s' % count]))
        count = count+1
    return result

def get_vectors(model, corpus, size):
    vecs = np.zeros((len(corpus), size))
    n = 0
    for i in corpus.index:
        prefix = 'all_' + str(i)
        vecs[n] = model.docvecs[prefix]
        n = n+1
    return vecs

Uncomment the function below for clean the data

In [61]:
#clean_data_set() 
df2 = pd.read_csv('cleaned_data_set.csv',encoding='latin1')
show_data_set_stats(df2)

Number of data by class
Class 1 --> 110
Class 2 --> 685
Class 3 --> 4227
Class 4 --> 1441
Class 5 --> 458


## Split on test and trainning data

In [62]:
# df3 = pd.read_csv('cleaned_data_set.csv', encoding="latin1")
x = df2.text
y = df2.sentiment

x_train = x[:5000]
y_train = y[:5000]
y_train_NN = oneHotVector(data=y_train,numClasses=5)


x_validation = x[-1921:]
y_validation = y[-1921:]
y_validation_NN = oneHotVector(data=y_validation,numClasses=5)

# Compare two models

In [63]:
tfidfVectorizer = TfidfVectorizer()
cntVectorizer = CountVectorizer()

#Set parameters for bigram and trigram

# tfidfVectorizer.set_params(ngram_range=(1,2))
# cntVectorizer.set_params(ngram_range=(1,2))

x_train_tfidVectorized = tfidfVectorizer.fit_transform(x_train).toarray()
x_validation_tfidVectorized = tfidfVectorizer.transform(x_validation).toarray()

x_train_cntVectorized = cntVectorizer.fit_transform(x_train).toarray()
x_validation_cntVectorized = cntVectorizer.transform(x_validation).toarray()

## Get spplited version of each tweet

We used the whole data set for training  Doc2Vec. You cand find the complete document [here](https://arxiv.org/pdf/1607.05368.pdf).

> We use all documents in the development and test
set (and potentially more background documents,
where  explicitly  mentioned)  to  train
doc2vec
.
Our rationale for this is that the
doc2vec
training
is  completely  unsupervised,
>
> -- <cite>Jey Han Lau and Timothy Baldwin</cite>

In [64]:
all_x_w2v = labelize_tweets_ug(df2.text, 'all')

## Implementing DBOW (Distributed Bag Of Words)

In [65]:
%%capture

cores = multiprocessing.cpu_count()
model_ug_dbow = Doc2Vec(dm=0,size=400, negative=5, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
# Build the vocabulary using labelized tweets
model_ug_dbow.build_vocab([x for x in tqdm(all_x_w2v)])

for epoch in range(30):
    model_ug_dbow.train(utils.shuffle([x for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)
    model_ug_dbow.alpha -= 0.002
    model_ug_dbow.min_alpha = model_ug_dbow.alpha
    

# get vectors from the labelized tweets
train_vecs_dbow = get_vectors(model_ug_dbow, x_train, 400)
validation_vecs_dbow = get_vectors(model_ug_dbow, x_validation, 400)

## Implementing DMC (Distributed Memory Concatenation)

Parameters taken from [here](http://proceedings.mlr.press/v32/le14.pdf).


In [78]:
%%capture

cores = multiprocessing.cpu_count()
model_ug_dmc = Doc2Vec(dm=1, dm_concat=1, size=400, window=10, negative=5, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_ug_dmc.build_vocab([x for x in tqdm(all_x_w2v)])

for epoch in range(30):
    model_ug_dmc.train(utils.shuffle([x for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)
    model_ug_dmc.alpha -= 0.002
    model_ug_dmc.min_alpha = model_ug_dmc.alpha
train_vecs_dmc = get_vectors(model_ug_dmc, x_train, 400)
validation_vecs_dmc = get_vectors(model_ug_dmc, x_validation, 400)

clf = LogisticRegression()
clf.fit(train_vecs_dmc, y_train)
clf.score(validation_vecs_dmc, y_validation)

100%|██████████| 6921/6921 [00:00<00:00, 815781.76it/s]


0.5653305570015617

In [99]:
def get_concat_vectors(model1,model2, corpus, size):
    vecs = np.zeros((len(corpus), size))
    n = 0
    for i in corpus.index:
        prefix = 'all_' + str(i)
        vecs[n] = np.append(model1.docvecs[prefix],model2.docvecs[prefix])
        n += 1
    return vecs

train_vecs_dbow_dmc = get_concat_vectors(model_ug_dbow,model_ug_dmc, x_train, 800)
validation_vecs_dbow_dmc = get_concat_vectors(model_ug_dbow,model_ug_dmc, x_validation, 800)

clf = LogisticRegression()
clf.fit(train_vecs_dbow_dmc, y_train)
clf.score(validation_vecs_dbow_dmc, y_validation)

0.6007287870900573

In [100]:
clf = LogisticRegression()
printStats(clf,'vecs_dbow',train_vecs_dbow_dmc, y_train,validation_vecs_dbow_dmc, y_validation)
printStats(clf,'tfidVectorized',x_train_tfidVectorized, y_train,x_validation_tfidVectorized, y_validation)
printStats(clf,'cntVectorized',x_train_cntVectorized, y_train,x_validation_cntVectorized, y_validation)

__________________________________________________________________________________
name		time	accuracy
vecs_dbow	4.34s	60.07	
__________________________________________________________________________________
name		time	accuracy
tfidVectorized	0.26s	59.86	
__________________________________________________________________________________
name		time	accuracy
cntVectorized	0.70s	60.49	


# Build NN for _doc2vec_

## Parameters

In [107]:
# Parameters
learning_rate = 0.001
training_epochs = 30
batch_size = 100
display_step = 1

# Network Parameters
n_hidden_1 = 256 # 1st layer number of features
n_hidden_2 = 256 # 2nd layer number of features
n_input = 800  # 800 inputs
n_classes = 5 # 5 classes 

## Graph


In [101]:
x = tf.placeholder("float", [None, n_input]) 
y = tf.placeholder("float", [None, n_classes])

# Create model
def multilayer_perceptron(x, weights, biases):

    print( 'x:', x.get_shape(), 'W1:', weights['h1'].get_shape(), 'b1:', biases['b1'].get_shape())        
    # Hidden layer with linear activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1']) #(x*weights['h1']) + biases['b1']
    layer_1 = tf.nn.relu(layer_1)

    # Hidden layer with linear activation
    print( 'layer_1:', layer_1.get_shape(), 'W2:', weights['h2'].get_shape(), 'b2:', biases['b2'].get_shape())        
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2']) # (layer_1 * weights['h2']) + biases['b2'] 
    layer_2 = tf.nn.relu(layer_2)

    # Output layer with Softmax activation
    print( 'layer_2:', layer_2.get_shape(), 'W3:', weights['out'].get_shape(), 'b3:', biases['out'].get_shape())        
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out'] # (layer_2 * weights['out']) + biases['out']   
    out_layer = tf.nn.softmax(out_layer)
    print('out_layer:',out_layer.get_shape())

    return out_layer


In [102]:
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),    #800x256
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])), #256x256
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))  #256x5
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),             #256x1
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),             #256x1
    'out': tf.Variable(tf.random_normal([n_classes]))              #5x1
}

In [103]:
# Construct model
pred = multilayer_perceptron(x, weights, biases)

x: (?, 800) W1: (800, 256) b1: (256,)
layer_1: (?, 256) W2: (256, 256) b2: (256,)
layer_2: (?, 256) W3: (256, 5) b3: (5,)
out_layer: (?, 5)


## Loss function, and Optimizer

In [104]:
# Cross entropy loss function
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred, labels=y))

# On this case we choose the AdamOptimizer
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Launch NN

In [105]:
# Initializing the variables
init = tf.global_variables_initializer()

def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[i] for i in idx]
    labels_shuffle = [labels[i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(y_train_NN.size/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = next_batch(batch_size,train_vecs_dbow_dmc,y_train_NN)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print ("Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    # To keep sizes compatible with model
    print ("Accuracy:", accuracy.eval({x: validation_vecs_dbow_dmc, y: y_validation_NN}))

Epoch: 0001 cost= 1.374258177
Epoch: 0002 cost= 1.291216628
Epoch: 0003 cost= 1.282853559
Epoch: 0004 cost= 1.295923735
Epoch: 0005 cost= 1.283388199
Epoch: 0006 cost= 1.281021480
Epoch: 0007 cost= 1.283265894
Epoch: 0008 cost= 1.285428069
Epoch: 0009 cost= 1.280745127
Epoch: 0010 cost= 1.285867087
Epoch: 0011 cost= 1.282388464
Epoch: 0012 cost= 1.277082780
Epoch: 0013 cost= 1.276581389
Epoch: 0014 cost= 1.277192652
Epoch: 0015 cost= 1.281592627
Epoch: 0016 cost= 1.277672629
Epoch: 0017 cost= 1.281152615
Epoch: 0018 cost= 1.274752625
Epoch: 0019 cost= 1.279632619
Epoch: 0020 cost= 1.271712624
Epoch: 0021 cost= 1.278912617
Epoch: 0022 cost= 1.276392622
Epoch: 0023 cost= 1.277952633
Epoch: 0024 cost= 1.278592623
Epoch: 0025 cost= 1.274512617
Epoch: 0026 cost= 1.272792639
Epoch: 0027 cost= 1.274192628
Epoch: 0028 cost= 1.277272618
Epoch: 0029 cost= 1.277432615
Epoch: 0030 cost= 1.282072614
Optimization Finished!
Accuracy: 0.56533056


# Build NN for _CountVectorizer_ and _TF-IDF Vectorizer_

In [106]:
 # Parameters
learning_rate = 0.002
training_epochs = 10
batch_size = 100
display_step = 1

# Network Parameters
n_hidden_1 = 256 # 1st layer number of features
n_hidden_2 = 256 # 2nd layer number of features
n_input = 8129 # 8129 inputs
n_classes = 5 # 5 classes 
x = tf.placeholder("float", [None, n_input]) 
y = tf.placeholder("float", [None, n_classes])

# Create model
def multilayer_perceptron(x, weights, biases):

    print( 'x:', x.get_shape(), 'W1:', weights['h1'].get_shape(), 'b1:', biases['b1'].get_shape())        
    # Hidden layer with linear activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1']) #(x*weights['h1']) + biases['b1']
    layer_1 = tf.nn.relu(layer_1)

    # Hidden layer with linear activation
    print( 'layer_1:', layer_1.get_shape(), 'W2:', weights['h2'].get_shape(), 'b2:', biases['b2'].get_shape())        
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2']) # (layer_1 * weights['h2']) + biases['b2'] 
    layer_2 = tf.nn.relu(layer_2)

    # Output layer with softmax activation
    print( 'layer_2:', layer_2.get_shape(), 'W3:', weights['out'].get_shape(), 'b3:', biases['out'].get_shape())        
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out'] # (layer_2 * weights['out']) + biases['out']    
    out_layer = tf.nn.softmax(out_layer)
    print('out_layer:',out_layer.get_shape())

    return out_layer

weights = {
'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),    #8129x256
'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])), #256x256
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))  #256x5
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),             #256x1
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),             #256x1
    'out': tf.Variable(tf.random_normal([n_classes]))              #5x1
}
# Construct model
pred = multilayer_perceptron(x, weights, biases)
# Cross entropy loss function
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred, labels=y))

# On this case we choose the AdamOptimizer
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

x: (?, 8129) W1: (8129, 256) b1: (256,)
layer_1: (?, 256) W2: (256, 256) b2: (256,)
layer_2: (?, 256) W3: (256, 5) b3: (5,)
out_layer: (?, 5)


# Launch CountVectorizer NN

In [107]:
# Initializing the variables
init = tf.initialize_all_variables()

def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[i] for i in idx]
    labels_shuffle = [labels[i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(y_train_NN.size/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = next_batch(batch_size,x_train_cntVectorized,y_train_NN)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print ("Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    # To keep sizes compatible with model
    print ("Accuracy:", accuracy.eval({x: x_validation_cntVectorized, y: y_validation_NN}))

Epoch: 0001 cost= 1.318430568
Epoch: 0002 cost= 1.277712661
Epoch: 0003 cost= 1.273192630
Epoch: 0004 cost= 1.271032627
Epoch: 0005 cost= 1.280632627
Epoch: 0006 cost= 1.280912619
Epoch: 0007 cost= 1.278752619
Epoch: 0008 cost= 1.277312613
Epoch: 0009 cost= 1.273312613
Epoch: 0010 cost= 1.273752627
Optimization Finished!
Accuracy: 0.5663717


# Launch TF-IDF Vectorizer NN

In [ ]:
# Initializing the variables
init = tf.initialize_all_variables()

def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[i] for i in idx]
    labels_shuffle = [labels[i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(y_train_NN.size/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = next_batch(batch_size,x_train_tfidVectorized,y_train_NN)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print ("Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    # To keep sizes compatible with model
    print ("Accuracy:", accuracy.eval({x: x_validation_tfidVectorized, y: y_validation_NN}))

Epoch: 0001 cost= 1.292709162
Epoch: 0002 cost= 1.275792626
Epoch: 0003 cost= 1.277512617
Epoch: 0004 cost= 1.272632621
Epoch: 0005 cost= 1.271072628
Epoch: 0006 cost= 1.275432628
Epoch: 0007 cost= 1.272912616
Epoch: 0008 cost= 1.280192622
Epoch: 0009 cost= 1.273832618
Epoch: 0010 cost= 1.279632617
Optimization Finished!
